In [ ]:
##propose: to match short covariates names with reference(long covariates names)

import numpy as np
import pandas as pd
import re
import csv
##Input data
df = read_sas('short.sas7dat',format = 'sas7dat')
sn_list = df['covariate'].str.decode(encoding = 'UTF-8')
longname = pd.read_excel('reference.xlsx',sheet_name = "logistic models",header = 2)
longname = longname['Variable'].dropna()

def find_most_match(short_name,reference):
    #race
    if short_name[0].lower() in ['w','f']:
        return link_race(short_name,reference,['W','F','w','f'])
    if short_name[0].lower() in ['m']:
        return link_race(short_name,reference,['M','m'])
    
    #hcc 
    if re.search(r'hcc',short_name):
        return hcc(short_name, reference)
    
    #ccs
    if re.search(r'ccs',short_name):
        return ccs(short_name,reference)
    
    #acute history stays
    if re.search(r'acute',short_name) or re.search(r'stay',short_name):
        return stay(short_name,reference)
    
    print('not find it in the reference, please check the category')
    return None

def link_race(short_name,reference,pref):
    s_age = re.findall('\d+_',s_name)
    if s_age:
        s_age = s_age[0][:-1]
    else:
        s_age = re.findall('\d+_',s_name)[0]
        
    gender = [cov for cov in reference if cov.strip()[0] in pref]
    for cov in gender:
        l_age = re.findall('\d+_',cov)
        if l_age:
            l_age = l_age[0][:-1]
        else:
            l_age = re.findall('\d+_',cov)[0]
        if s_age == l_age:
            return cov
    print('not find it in long name')
    return None

def hcc(short_name,reference):
    hcc_llist = [cov for cov in reference if re.search(r'hcc',cov,re.IGNORECASE)]
    if re.findall('\d+',short_name):
        sid = re.findall('\d+',short_name)[0][:-1]
    else:
        sid = re.findall('\d+',short_name)[0]
    for cov in hcc_llist:
        lid = re.findall('hcc\d+',cov,re.IGNORECASE)[0][3:]
        if sid ==lid:
            return cov
    print('not find in long name')
    return None

def ccs(short_name,reference):
    ccs_list = [cov for cov in reference if re.search(r'\(\d+.*\)',cov,re.IGNORECASE)]
    sid = re.findall('\d+',short_name)[0]
    for cov in ccs_list:
        lid = re.findall('\d+',cov)[0]
        if sid == lid:
            return cov
    print('not find in long name')
    return None  


def stay(short_name,reference):
    l = [cov for cov in reference if re.search(r'acute history',cov,re.IGNORECASE)]
    sid = re.findall('\d+',short_name)[0]
    for cov in l:
        lid = re.findall('\d+',cov)[0]
        if sid == lid:
            return cov
    print('not find in long name')
    return None  
   
#test case: F_85
#test case: m18_34

res = {x:'' for x in sn_list}
#sn_list should be lower case
for srt in sn_list:
    res[srt] = find_most_match(srt,longname)

with open('result.csv','w',newline = '') as f:
    writer = csv.writer(f)
    for i,j in res.items():
        writer.writerow([i,j])
f.close()
    
    
    
